In [16]:
class new_sorted:
    def __init__(self, arr, key, reverse = False):
        # Check if list in inputted
        if type(arr) == list and bool(reverse):
            self.lst = arr
            self.key = k
            self.reversed = reverse
        
        # Let user know list not inputted for arr
        elif type(arr) != list:
            raise ValueError('new_sorted only accepts lists as attributes')
        
        # Let user know they entered wrong value for reverse
        elif bool(reverse) == False:
            raise ValueError('new_sorted only accepts lists as attributes')